In [29]:
from SRC.foursquare import exploreForesquare, searchForesquare
from SRC.geofunc import *
from SRC.mongofunc import connectMongoColl
from SRC.goog import getLocation

import pandas as pd
import geopandas as gpd
from pymongo import MongoClient
import numpy as np
from geopy import distance
from haversine import haversine
# harvesine (lat,long) --> dist in km.

#Carto
from cartoframes.viz import Map, Layer
from cartoframes.viz.helpers import size_continuous_layer
from cartoframes.viz.widgets import histogram_widget

# Geo_Apis
### For the best value, let's see the situation of the rest of the filters
---

In [2]:
coord = [37.772323,-122.214897]

## 1 | PARTY

In [3]:
party = searchForesquare(ID="party",limit=15,distance=500,latitude=37.772323,longitude=-122.214897)

In [4]:
dfparty = pd.DataFrame(party["response"]["venues"])
dfparty

,id,name,location,categories,referralId,hasPerk
0,4e7e9edf8231243aa62dff89,Party Spot,"{'lat': 37.776357, 'lng': -122.216319, 'labele...","[{'id': '4bf58dd8d48988d11a941735', 'name': 'O...",v-1581404698,False
1,4f40ad70e4b01e824ddddb6f,12th Street Lounge,"{'address': '3700 E 12th St Fl 2', 'lat': 37.7...","[{'id': '4bf58dd8d48988d1d8941735', 'name': 'G...",v-1581404698,False
2,4ae687dff964a52013a721e3,La Frontera Nightclub,"{'address': '4481 International Blvd', 'lat': ...","[{'id': '4bf58dd8d48988d118941735', 'name': 'D...",v-1581404698,False
3,4d42444a33268cfaa0465701,Mel's Loft,"{'address': '4001 San Leandro St', 'crossStree...","[{'id': '4bf58dd8d48988d1d4941735', 'name': 'S...",v-1581404698,False
4,4eaa7989e3000d1f05738cbb,The Bricks,"{'lat': 37.773972, 'lng': -122.22118701487776,...","[{'id': '4bf58dd8d48988d1d6941735', 'name': 'S...",v-1581404698,False


In [5]:
loc_party = []
for i in dfparty.location:
    loc_party.append([float(i['lat']),float(i['lng'])])

In [6]:
loc_party

[[37.776357, -122.216319],
 [37.774036189028635, -122.22128921920071],
 [37.771166, -122.213862],
 [37.771209000000006, -122.21972099999999],
 [37.773972, -122.22118701487776]]

In [7]:
names = []
for i in dfparty.name:
    names.append(i)

names
    

['Party Spot',
 '12th Street Lounge',
 'La Frontera Nightclub',
 "Mel's Loft",
 'The Bricks']

In [8]:
part =  dict(zip(names,loc_party))

In [10]:
dfparty = pd.DataFrame(part)
dfparty = dfparty.T.reset_index().rename(columns={"index":"name",0: "latitude", 1: "longitude"})
gdfparty = gpd.GeoDataFrame(dfparty, geometry=gpd.points_from_xy(dfparty.longitude, dfparty.latitude))
display(gdfparty)

,name,latitude,longitude,geometry
0,Party Spot,37.776357,-122.216319,POINT (-122.21632 37.77636)
1,12th Street Lounge,37.774036,-122.221289,POINT (-122.22129 37.77404)
2,La Frontera Nightclub,37.771166,-122.213862,POINT (-122.21386 37.77117)
3,Mel's Loft,37.771209,-122.219721,POINT (-122.21972 37.77121)
4,The Bricks,37.773972,-122.221187,POINT (-122.22119 37.77397)


In [ ]:
# Add row of likes and comment

## 2 | Starbucks

In [11]:
dfst = pd.read_csv("INPUT/starbucks.csv")
dfst.head()

,Brand,Store Number,Name,Ownership Type,Facility ID,Features - Products,Features - Service,Features - Stations,Food Region,Venue Type,...,Street Line 1,Street Line 2,City,State,Zip,Country,Coordinates,Latitude,Longitude,Insert Date
0,Starbucks,11854,Epping Main St,Company Owned,18228,"Lunch, Oven-warmed Food, Reserve Amenity","Starbucks Card Mobile, Wireless Hotspot",Drive-Through,9999,Unknown,...,96 Calef Highway,NaN,Epping,NH,03042-2224,US,"(43.031873, -71.073203)",43.031873,-71.073203,06/22/2012 06:17:10 PM
1,Starbucks,79420,Stop & Shop-Exeter #204,Licensed,9956,NaN,Wireless Hotspot,NaN,9999,Unknown,...,139 Portsmouth Ave,NaN,Exeter,NH,03833-2105,US,"(42.989763, -70.932044)",42.989763,-70.932044,06/22/2012 06:17:10 PM
2,Starbucks,3872,Target Greenland T-2530,Licensed,10933,NaN,Starbucks Card Mobile,NaN,9999,Unknown,...,1450 Greenland Rd,NaN,Greenland,NH,03840-2438,US,"(43.04871, -70.816345)",43.048710,-70.816345,06/22/2012 06:17:10 PM
3,Starbucks,13448,"Seabrook, Lafayette Rd",Company Owned,16980,"Lunch, Oven-warmed Food, Reserve Amenity, Sire...","Starbucks Card Mobile, Wireless Hotspot",Drive-Through,9999,Unknown,...,271 Lafayette Rd,NaN,Seabrook,NH,03874-4512,US,"(42.882128, -70.86808)",42.882128,-70.868080,06/22/2012 06:17:10 PM
4,Starbucks,10325,"Portsmouth, Woodbury Avenue",Company Owned,17269,"Lunch, Oven-warmed Food, Reserve Amenity","Starbucks Card Mobile, Wireless Hotspot",Drive-Through,9999,Unknown,...,1855 Woodbury Avenue,NaN,Portsmouth,NH,03801-3228,US,"(43.092318, -70.792327)",43.092318,-70.792327,06/22/2012 06:17:10 PM


In [12]:
dfst["location"] = dfst[["Latitude","Longitude"]].apply(lambda x:asGeoJSON(x.Latitude,x.Longitude), axis=1)
dfst = dfst.loc[:, ['Brand', 'Name', "State",'location']]
dfst.head()

,Brand,Name,State,location
0,Starbucks,Epping Main St,NH,"{'type': 'Point', 'coordinates': [-71.073203, ..."
1,Starbucks,Stop & Shop-Exeter #204,NH,"{'type': 'Point', 'coordinates': [-70.93204399..."
2,Starbucks,Target Greenland T-2530,NH,"{'type': 'Point', 'coordinates': [-70.816345, ..."
3,Starbucks,"Seabrook, Lafayette Rd",NH,"{'type': 'Point', 'coordinates': [-70.86808, 4..."
4,Starbucks,"Portsmouth, Woodbury Avenue",NH,"{'type': 'Point', 'coordinates': [-70.792327, ..."


In [13]:
val=[]
for i in dfst.location:
    if haversine(coord,i["coordinates"]) < 50:
        val.append("YES")
    else:
        val.append("NO")
        
dfst.loc[:, "near_point"] = val


In [14]:
dfst.to_json("OUTPUT/Starbucks.json", orient="records")
# $ mongoimport --db starbucks --collection starbucks --jsonArray --drop OUTPUT/Starbucks.json

In [15]:
db, collst = connectMongoColl('starbucks','starbucks')

In [16]:
result = collst.aggregate([
    {
        '$geoNear': {
            'near': {
                'type': 'Point', 
                'coordinates': [
                    -122.214897, 37.772323
                ]
            }, 
            'distanceField': 'dist.calculated', 
            'minDistance': 2, 
            'spherical': True
        }
    }
])

In [17]:
# COMO SE HACE ESTO!!!!!

In [18]:
db, collst_dist = connectMongoColl('starbucks','starbucks_dist')

In [19]:
filt = {"Brand":1,"State":1,"location":1}
starb = list(collst_dist.find({"dist.calculated":{"$lte":10000}},filt))
df = pd.DataFrame(starb)
df

""


In [20]:
# POR QUÉ NO FUNCIONA!

In [21]:
df = pd.read_csv("INPUT/starbucks.csv")
df = df.loc[:, ['Brand', 'Name', 'State','Latitude','Longitude']]
df = df.rename(columns={"Latitude":"latitude","Longitude":"longitude"})
df.head()

,Brand,Name,State,latitude,longitude
0,Starbucks,Epping Main St,NH,43.031873,-71.073203
1,Starbucks,Stop & Shop-Exeter #204,NH,42.989763,-70.932044
2,Starbucks,Target Greenland T-2530,NH,43.048710,-70.816345
3,Starbucks,"Seabrook, Lafayette Rd",NH,42.882128,-70.868080
4,Starbucks,"Portsmouth, Woodbury Avenue",NH,43.092318,-70.792327


In [22]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
gdf.head()

,Brand,Name,State,latitude,longitude,geometry
0,Starbucks,Epping Main St,NH,43.031873,-71.073203,POINT (-71.07320 43.03187)
1,Starbucks,Stop & Shop-Exeter #204,NH,42.989763,-70.932044,POINT (-70.93204 42.98976)
2,Starbucks,Target Greenland T-2530,NH,43.048710,-70.816345,POINT (-70.81634 43.04871)
3,Starbucks,"Seabrook, Lafayette Rd",NH,42.882128,-70.868080,POINT (-70.86808 42.88213)
4,Starbucks,"Portsmouth, Woodbury Avenue",NH,43.092318,-70.792327,POINT (-70.79233 43.09232)


In [23]:
def get_distance(row):
    '''
    Esta función calcula la distancia en km entre el punto seleccionado y las coordenadas que pasamos como parámetro
    '''
    coord = (37.772323,-122.214897)
    dist = distance(coord, (row.latitude, row.longitude)).km
    return round(dist, 2)

In [24]:
gdf2 = gdf.drop(columns=["Brand","Name","State"])
gdf2.head()

,latitude,longitude,geometry
0,43.031873,-71.073203,POINT (-71.07320 43.03187)
1,42.989763,-70.932044,POINT (-70.93204 42.98976)
2,43.048710,-70.816345,POINT (-70.81634 43.04871)
3,42.882128,-70.868080,POINT (-70.86808 42.88213)
4,43.092318,-70.792327,POINT (-70.79233 43.09232)


In [25]:
gdf2['distance'] = gdf2.apply(get_distance, axis=1)
gdf2.head()

TypeError: ("'module' object is not callable", 'occurred at index 0')

In [ ]:
# filtrar gdf por los que están a menos de 5/10 km.

In [58]:
#Map(Layer(gdf))

## 3 | School

In [30]:
school = getLocation(radius = 2000)

In [52]:
dschool = pd.DataFrame(school["results"])

In [53]:
dschool.head()

,geometry,icon,id,name,photos,place_id,plus_code,rating,reference,scope,types,user_ratings_total,vicinity,opening_hours
0,"{'location': {'lat': 37.7645032, 'lng': -122.2...",https://maps.gstatic.com/mapfiles/place_api/ic...,d69e15e88cd3a2d0a14b6cfc80db303a68e681d4,Edison Elementary School,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJSTHeApGGj4ARIEJSgvDoX0o,"{'compound_code': 'QQ78+R2 Alameda, California...",3.2,ChIJSTHeApGGj4ARIEJSgvDoX0o,GOOGLE,"[school, point_of_interest, establishment]",6,"2700 Buena Vista Ave, Alameda",NaN
1,"{'location': {'lat': 37.7755994, 'lng': -122.2...",https://maps.gstatic.com/mapfiles/place_api/ic...,3b5324e357d2539e884c89065f0bb86cc5190c7e,Aspire Public Schools,NaN,ChIJFdkP-WaGj4ARpenFXp8fOm0,"{'compound_code': 'QQGQ+67 Oakland, California...",1.0,ChIJFdkP-WaGj4ARpenFXp8fOm0,GOOGLE,"[school, point_of_interest, establishment]",1,"1936 Courtland Ave, Oakland",NaN
2,"{'location': {'lat': 37.7793766, 'lng': -122.2...",https://maps.gstatic.com/mapfiles/place_api/ic...,76d76020e2dd748df52bd9b50f7fe953e6043c5b,International Community School,NaN,ChIJx-qh0fGGj4AR--nZOdZ7AKE,"{'compound_code': 'QQH9+QQ Oakland, California...",4.2,ChIJx-qh0fGGj4AR--nZOdZ7AKE,GOOGLE,"[school, point_of_interest, establishment]",6,"2825 International Blvd, Oakland",NaN
3,"{'location': {'lat': 37.7787755, 'lng': -122.2...",https://maps.gstatic.com/mapfiles/place_api/ic...,97d1fcb3538f5f69fa13223b8c9b21587f60cf76,St. Elizabeth Elementary & Middle Schools,NaN,ChIJG3WhPPSGj4ARNpww7ttbA-0,"{'compound_code': 'QQHG+GF Oakland, California...",4.8,ChIJG3WhPPSGj4ARNpww7ttbA-0,GOOGLE,"[school, point_of_interest, establishment]",11,"1516 33rd Ave, Oakland",{'open_now': True}
4,"{'location': {'lat': 37.7904349, 'lng': -122.2...",https://maps.gstatic.com/mapfiles/place_api/ic...,4854462f7c679ae1d141e81eb8b1ed5c3a6e2dce,Roosevelt Middle School,NaN,ChIJlftz_eGGj4ARTfEreQssTUM,"{'compound_code': 'QQR7+55 Oakland, California...",3.6,ChIJlftz_eGGj4ARTfEreQssTUM,GOOGLE,"[secondary_school, school, point_of_interest, ...",34,"1926 E 19th St, Oakland",NaN


In [54]:
lst_school=[]
for i in dschool.geometry:
    lst_school.append(i["location"])

In [61]:
dschool = pd.DataFrame(lst_school)
dschool.rename(columns={"lat":'latitude','lng':"longitude"})

,latitude,longitude
0,37.764503,-122.234896
1,37.775599,-122.211805
2,37.779377,-122.230619
3,37.778776,-122.223856
4,37.790435,-122.237026
5,37.775823,-122.224785
6,37.757445,-122.233304
7,37.785865,-122.219579
8,37.774563,-122.208907
9,37.779545,-122.213285


In [62]:
gdfschool = gpd.GeoDataFrame(dschool, geometry=gpd.points_from_xy(df.longitude, df.latitude))
gdfschool.head()

ValueError: Length of values does not match length of index